In [1]:
import numpy as np
from scipy.sparse import random
from scipy import stats
from ddeint import ddeint
import matplotlib.pyplot as plt
import math
import pandas as pd
from scipy.sparse import csc_matrix

In [9]:
# Reservoir generating functions
def gen_matrix(shape, sparsity, sd=1, mean=0, loc_seed=100, val_seed=100, pdf="gaussian", seeded=True):
    
    def seeded_rvs_gauss(array_len):
            return stats.norm(loc=mean, scale=sd).rvs(random_state = val_seed, size=array_len)

    def seeded_rvs_uniform(array_len):
        return stats.uniform(loc=mean, scale=sd).rvs(random_state = val_seed, size=array_len)

    m = shape[0]
    n = shape[1]

    if seeded == True:
        
        if pdf == "gaussian":
            M = random(m, n, density=sparsity, random_state=loc_seed, data_rvs=seeded_rvs_gauss).A
            return M

        if pdf == "uniform":
            M = random(m, n, density=sparsity, random_state=loc_seed, data_rvs=seeded_rvs_uniform).A
            return M

        if pdf == "ones":
            M = random(m, n, density = sparsity, random_state=loc_seed, data_rvs=np.ones).A
            return M
        else: 
            print("No such pdf")
            
    elif seeded == False:
        
        if pdf == "gaussian":
            unseeded_rvs = stats.norm(loc=mean, scale=sd).rvs
            M = random(m, n, density=sparsity, data_rvs=unseeded_rvs).A
            return M

        if pdf == "uniform":
            unseeded_rvs = stats.uniform(loc=mean, scale=sd).rvs
            M = random(m, n, density=sparsity, data_rvs=unseeded_rvs).A
            return M

        if pdf == "ones":
            M = random(m, n, density = sparsity, data_rvs=np.ones).A
            return M
        else: 
            print("No such pdf")
            
    else:
        print("Seeded was neither true nor false")

def spectral_radius(M):
    max_abs_eigenvalue = -1
    eigenvalues, eigenvectors = np.linalg.eig(M)
    for eigenvalue in eigenvalues:
        if abs(eigenvalue) > max_abs_eigenvalue:
            max_abs_eigenvalue = abs(eigenvalue)
    return max_abs_eigenvalue

def spectral_radius_matrix(M, desired_spec_rad):
    M_sr = spectral_radius(M)
    if M_sr == 0:
        return M
    else:
        M = M*(desired_spec_rad/M_sr)
        return M
    
# ESN equations
def sigma(value):
    return np.tanh(value)

def state(x_prev, z_curr):
    z_curr = np.atleast_2d(z_curr)
    x_curr = sigma(np.matmul(A, x_prev) + gamma*np.matmul(C, z_curr) + s*zeta)
    return x_curr

def observation(x_curr, w):
    z_next = np.matmul(np.transpose(w), x_curr)
    return z_next

def listening(training_data, x_0, A, gamma, C, s, zeta):
    state_dict = {'all_states': None,
                  'last_state': None, 
                  'input_data': None}
    
    T = len(training_data)
    
    for t in range(0, T):
        if t == 0:
            x_curr = x_0
            X = np.array(x_curr)
            z_curr = training_data[t]
            Z = np.atleast_2d(np.array([z_curr]))
        else:
            x_curr = state(x_curr, z_curr)
            X = np.column_stack((X, x_curr))
            z_curr = training_data[t]
            Z = np.column_stack((Z, z_curr))

    state_dict['last_state'] = x_curr
    state_dict['all_states'] = X
    state_dict['input_data'] = Z
    
    return state_dict
    
def regression_sol(ld, state_dict, N):
    X = state_dict['all_states']
    Z = state_dict['input_data']
    
    X = X[:, 1000:]
    Z = Z[:, 1000:]

    X_transpose = X.transpose()
    Z_transpose = Z.transpose()
    XZ_transpose = np.matmul(X, Z_transpose)
    
    inverse_term = np.linalg.inv(np.matmul(X, X_transpose) + ld*np.identity(N))
    W_best = np.matmul(inverse_term, XZ_transpose)
    
    return W_best

# Data generation function
def generate_MG_data(init_val, tau, a, b, n):
    def MackeyGlass(z, t):
        return (a * z(t - tau)) / (1 + z(t - tau)**n) - b*z(t)

    def initial(t):
        return init_val

    time_space = np.arange(0, 5000, 1)

    z_solution_new = ddeint(MackeyGlass, initial, time_space)
    z_solution_new  = [ data_pt[0] for data_pt in z_solution_new[1:] ]
    z_solution_new.insert(0, init_val)
    z_solution_new = [ np.tanh(unshifted_val-1) for unshifted_val in z_solution_new ]

    training_data = z_solution_new[0:3000]
    testing_data = z_solution_new[3000:]
    
    return training_data, testing_data

# Plotting functions
def state_plot(state_dict, plotwith_init, node=0):
    X = state_dict.get('all_states')
    if plotwith_init == True:
        state_plot, state_ax = plt.subplots(figsize=(20,5))
        state_ax.plot(X[node][:])
        state_ax.set_title('Plot of States at node {}'.format(node))
        state_ax.set_xlabel('time')
        state_ax.set_ylabel('state of node {}'.format(node))
        
        return (np.amin(X[node][:]), np.amax(X[node][:]))
    
    if plotwith_init == False:
        state_plot, state_ax = plt.subplots(figsize=(20,5))
        state_ax.plot(X[node][1000:])
        state_ax.set_title('Plot of States at node {}'.format(node))
        state_ax.set_xlabel('time')
        state_ax.set_ylabel('state of node {}'.format(node))
    
        return (np.amin(X[node][1000:]), np.amax(X[node][1000:]))
                
def hist_accuracy_plot(actuals, predictions, with_bars=False):
    if with_bars == False:
        sns.kdeplot(actuals, label='actual', shade=True, color='red')
        sns.kdeplot(predictions, label='prediction', shade=True, color='skyblue')
        plt.legend()
        
    if with_bars == True:
        sns.histplot(actuals, label='actual', color='red', kde=True)
        sns.histplot(predictions, label='prediction', color='skyblue', kde=True)
        plt.legend()

In [13]:
def pf0_px(x): # (N,1) vector
    return csc_matrix(weight_result.transpose())

def outer_term(x, z):
    csc_A = csc_matrix(A)
    csc_C = csc_matrix(C)
    csc_zeta = csc_matrix(zeta)
    
    tanh_term = np.tanh(csc_A*x + gamma*csc_C*z + s*csc_zeta)
    
    return csc_matrix(1 - np.power(tanh_term, 2))

def pf_px(x, z): # (N,1) vector and scalar
    outer = outer_term(x, z)
    return outer.multiply(A)
    
def pf_pz(x, z): # (N,1) vector and scalar
    outer = outer_term(x, z)
    return outer.multiply(C)

def gram_schmidt(u):
    alpha = np.zeros(shape=(N,))
    for j in range(N):
        for k in range(j):
            u[: , j] = u[: , j] - np.dot(u[: , k], u[: , j]) * u[: , k]
        alpha[j] = np.linalg.norm(u[: , j])
        u[:, j] = u[: , j] / alpha[j]
    return u, alpha

In [14]:
data = generate_MG_data(1.2, 17, 0.2, 0.1, 10)

warmup_data = data[0]

In [15]:
N = 1000
ld = 10**(-14)
gamma = 0.8
s = 0.2
C = np.loadtxt('../1. Cleaned MG ESN Code/Parameters/C').reshape(N,1)
A = np.loadtxt('../1. Cleaned MG ESN Code/Parameters/A').reshape(N,N)
zeta = np.loadtxt('../1. Cleaned MG ESN Code/Parameters/zeta').reshape(N,1)
x_0 = np.zeros(shape=(N,1), dtype=float)

state_dict = listening(warmup_data, x_0, A, gamma, C, s, zeta)
weight_result = regression_sol(ld, state_dict, N) 

In [10]:
T_max = 100
delta = 10**(-6)
h = 1   # needs to correspond to the step size in the data set
tau = 17

In [17]:
Delta = delta * np.identity(N)

Lambda = np.zeros(shape=(N, ))

x_curr = state_dict['last_state']
z_curr = state_dict['input_data'][0, -1]

J0 = pf0_px(x_curr)

for t in range(0, T_max):
    
    x_curr = state(x_curr, z_curr)
    z_curr = observation(x_curr, weight_result) 

    J1 = pf_px(x_curr, z_curr) 
    J2 = pf_pz(x_curr, z_curr)
    J = J1 + J2*J0
    Delta = J*Delta
    
    Delta, alpha = gram_schmidt(Delta)

    Lambda = Lambda + np.log(alpha)

Lambda = Lambda/(T*tau)

KeyboardInterrupt: 

In [19]:
# Kaplan-Yorke dimension

Lambda_sorted = sorted(Lambda, reverse=True)

sum_Lambda = 0

D_KY = 0

for Lambda_id in range(0, len(Lambda_sorted)):
    sum_Lambda = sum_Lambda + Lambda_sorted[Lambda_id]
    if sum_Lambda < 0:
        D_KY = Lambda_id - 1 - ((sum_Lambda - Lambda_sorted[Lambda_id])/Lambda_sorted[Lambda_id])
        break
        
D_KY = D_KY + 1

In [20]:
D_KY

0.0